# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [3]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out nutirition advice.   
    You give concise answers.
    """
)

Let's execute the Agent:

In [4]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are generally healthy as part of a balanced diet.
    
    Key points:
    - Nutrients: good source of potassium, vitamin C, vitamin B6, and fiber.
    - Calories: about 90–105 kcal for a medium banana.
    - Carbs/sugar: mainly natural sugars with fiber; ripe bananas have more sugar.
    - Health benefits: supports heart health, digestion, and energy; can aid satiety when paired with protein or fat.
    - Considerations: portion size for diabetes or blood sugar concerns; choose ripe or unripe based on taste and digestion preference (unripe has more resistant starch).
    
    Bottom line: a convenient, nutritious snack or addition to meals when eaten in moderation.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [5]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are generally very healthy.

Key nutrients per medium banana (about 118 g):
- Potassium: supports blood pressure and heart function
- Vitamin B6: helps metabolism and brain function
- Vitamin C: antioxidant and immune support
- Fiber: aids digestion (about 3 g)
- Some resistant starch in less ripe ones

Benefits:
- Quick energy source (natural sugars and carbs)
- Good for athletes and recovery
- May support digestion and heart health

Considerations:
- Moderation if watching sugar intake (ripe bananas are higher in sugar)
- People with kidney disease may need potassium limits
- If you have latex-fruit syndrome, check for potential cross-reactivity

Storage: room temp to ripen; refrigerating can slow ripening but may affect texture.

Bottom line: a healthy, convenient snack or add-on to meals.

_Good Job!_